In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [2]:
# 1. Iniciar SparkSession y Crear Datos de Ejemplo
spark = SparkSession.builder.appName("PurchasePrediction").getOrCreate()

data = [
    (1, 10, 5, 3, "electronica", 1), (2, 2, 25, 8, "hogar", 1),
    (3, 30, 8, 1, "moda", 0), (4, 1, 15, 12, "electronica", 1),
    (5, 45, 2, 0, "libros", 0), (6, 8, 18, 6, "hogar", 1),
    (7, 90, 1, 0, "moda", 0), (8, 5, 30, 10, "electronica", 1),
    (9, 60, 3, 1, "hogar", 0), (10, 15, 12, 4, "libros", 0)
]
columns = ["id_cliente", "dias_ultima_visita", "paginas_vistas", "compras_mes_anterior", "ultimo_producto_visto", "comprara_prox_dias"]
df = spark.createDataFrame(data, columns)

# La columna objetivo debe llamarse "label" para los evaluadores de MLlib
df = df.withColumnRenamed("comprara_prox_dias", "label")

# 2. Definir Pipeline de Transformación y Vectorización
# Columnas categóricas y numéricas
categorical_cols = ["ultimo_producto_visto"]
numerical_cols = ["dias_ultima_visita", "paginas_vistas", "compras_mes_anterior"]

# Etapa 1: Convertir strings a índices numéricos
string_indexer = StringIndexer(inputCols=categorical_cols, outputCols=[c + "_index" for c in categorical_cols])
# Etapa 2: One-Hot Encoding
one_hot_encoder = OneHotEncoder(inputCols=string_indexer.getOutputCols(), outputCols=[c + "_ohe" for c in categorical_cols])
# Etapa 3: Ensamblar todas las características en un solo vector
assembler_inputs = numerical_cols + one_hot_encoder.getOutputCols()
vector_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# Se crea un pipeline solo para el preprocesamiento
preprocessing_pipeline = Pipeline(stages=[string_indexer, one_hot_encoder, vector_assembler])

# 3. Implementar el Modelo Supervisado (Random Forest)
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# 4. Dividir los datos
(training_data, test_data) = df.randomSplit([0.8, 0.2], seed=42)

In [3]:
# 5. Ajustar Hiperparámetros con Validación Cruzada
# Se define una grilla de parámetros para probar
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [10, 20])
             .addGrid(rf.maxDepth, [5, 10])
             .build())

# Se define el evaluador que medirá el rendimiento
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

# Se crea el CrossValidator
cv = CrossValidator(estimator=rf,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3) # Usar 3 folds para validación cruzada

# Se crea el Pipeline final que une el preprocesamiento y el modelo con CrossValidator
final_pipeline = Pipeline(stages=[preprocessing_pipeline, cv])

# Entrenar el pipeline completo. Esto ejecuta la validación cruzada para encontrar el mejor modelo.
pipeline_model = final_pipeline.fit(training_data)

In [4]:
# 6. Realizar Predicciones y Evaluar el Desempeño
predictions = pipeline_model.transform(test_data)

# Evaluar con AUC-ROC
roc_auc = evaluator.evaluate(predictions)

# Evaluar con Precisión-Recall
evaluator.setMetricName("areaUnderPR")
pr_auc = evaluator.evaluate(predictions)

# 7. Mostrar Resultados
print("\n--- Resultados de la Evaluación del Modelo ---")
print(f"Área bajo la curva ROC (AUC-ROC) en datos de prueba: {roc_auc:.4f}")
print(f"Área bajo la curva de Precisión-Recall (AUC-PR) en datos de prueba: {pr_auc:.4f}")

spark.stop()


--- Resultados de la Evaluación del Modelo ---
Área bajo la curva ROC (AUC-ROC) en datos de prueba: 1.0000
Área bajo la curva de Precisión-Recall (AUC-PR) en datos de prueba: 1.0000
